In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.9.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.9.* or later!')

In [2]:
%matplotlib inline

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

/home/linda/models/research/object_detection/utils/visualization_utils.py:27: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.
  import matplotlib; matplotlib.use('Agg')  # pylint: disable=multiple-statements


In [4]:
# What model to download.
MODEL_NAME = '/home/linda/OLIV/training/models/ssd_mobilenetv2_1_300/export/ssd_mobilenetv2_1_300_800000'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/home/linda/OLIV/training/data', 'label_map.pbtxt')

In [5]:
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')
    sess = tf.Session(graph=detection_graph)

In [6]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [7]:
import pickle 
fileObject = open('/home/linda/OLIV/training/data/filenames_id.p','rb')  
filenames_id = pickle.load(fileObject) 

image_names = []
count = 0 
for key in filenames_id:
    image_names.append(key)
    count += 1
print(count)

13304


In [8]:
PATH_TO_TEST_IMAGES_DIR = '/home/linda/OLIV/training/data/images/cooc2017'
#IMAGE_NAME = '000000253906.jpg'
#TEST_IMAGE_PATH = os.path.join(PATH_TO_TEST_IMAGES_DIR, IMAGE_NAME)

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)

In [9]:
def run_inference_for_single_image(image, sess, graph):
    #with graph.as_default():
        #with tf.Session() as sess:
    # Get handles to input and output tensors
    ops = graph.get_operations()
    all_tensor_names = {output.name for op in ops for output in op.outputs}
    tensor_dict = {}
    for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
            tensor_dict[key] = graph.get_tensor_by_name(tensor_name)
    if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(detection_masks_reframed, 0)
    image_tensor = graph.get_tensor_by_name('image_tensor:0')

    # Run inference
    output_dict = sess.run(tensor_dict,
                       feed_dict={image_tensor: np.expand_dims(image, 0)})

    # all outputs are float32 numpy arrays, so convert types as appropriate
    output_dict['num_detections'] = int(output_dict['num_detections'][0])
    output_dict['detection_classes'] = output_dict['detection_classes'][0].astype(np.uint8)
    output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
    output_dict['detection_scores'] = output_dict['detection_scores'][0]
    if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

In [10]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [11]:
'''class_labels = {'bottle': 44, 'person': 1, 'backpack': 27, 'handbag': 31, 
                'umbrella': 28, 'bowl': 51, 'cup': 47, 'clock': 85, 
                'knife': 49, 'spoon': 50, 'fork': 48, 'chair': 62, 
                'potted plant': 64, 'dining table': 67, 'keyboard': 76, 'mouse': 74, 
                'laptop': 73, 'cell phone': 77, 'scissors': 87, 'book': 84, 'monitor': 72}

class_ids = {44, 1, 27, 31, 28, 51, 47, 85, 49, 50, 48, 62, 64, 67, 76, 74, 73, 77, 87, 84, 72}''' 

class_labels = {'bottle': 1, 'person': 2, 'backpack': 3, 'handbag': 4, 
                'umbrella': 5, 'bowl': 6, 'cup': 7, 'clock': 8, 
                'knife': 9, 'spoon': 10, 'fork': 11, 'chair': 12, 
                'potted plant': 13, 'dining table': 14, 'keyboard': 15, 'mouse': 16, 
                'laptop': 17, 'cell phone': 18, 'scissors': 19, 'book': 20, 'monitor': 21}
class_ids = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21}

def write_to_json(output_dict, image_name, isFirst):
    i = 0 
    while i < output_dict['num_detections']:
        # "image_id":1, "category_id":18, "bbox": [x, y, width, height], "score":0.7
        if output_dict['detection_classes'][i] in class_ids and output_dict['detection_scores'][i] > 0:
            if isFirst:
                file.write("{\"image_id\": " + str(filenames_id[image_name])
                          + ", \"category_id\": "+ str(output_dict['detection_classes'][i])
                          + ", \"bbox\": [" + str(output_dict['detection_boxes'][i][1]*image.size[0]) + ", "
                          + str(output_dict['detection_boxes'][i][0]*image.size[1]) + ", "
                          + str((output_dict['detection_boxes'][i][3]-output_dict['detection_boxes'][i][1])*image.size[0]) + ", "
                          + str((output_dict['detection_boxes'][i][2]-output_dict['detection_boxes'][i][0])*image.size[1])
                          + "], \"score\": " + str(output_dict['detection_scores'][i]) + "}")
                isFirst = False
            else:
                file.write(",{\"image_id\": " + str(filenames_id[image_name])
                          + ", \"category_id\": "+ str(output_dict['detection_classes'][i])
                          + ", \"bbox\": [" + str(output_dict['detection_boxes'][i][1]*image.size[0]) + ", "
                          + str(output_dict['detection_boxes'][i][0]*image.size[1]) + ", "
                          + str((output_dict['detection_boxes'][i][3]-output_dict['detection_boxes'][i][1])*image.size[0]) + ", "
                          + str((output_dict['detection_boxes'][i][2]-output_dict['detection_boxes'][i][0])*image.size[1])
                          + "], \"score\": " + str(output_dict['detection_scores'][i]) + "}")
        i += 1

In [12]:
file = open("subsetCOCO_testRes_1_300.json", "w")
file.write("[")
first = True

from IPython.display import display, clear_output
num_iter = 0 

for im_name in image_names:
    clear_output(wait=True)
    num_iter += 1
    display(num_iter)
    #if num_iter > 10:
        #break
    test_image_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, im_name)
    image = Image.open(test_image_path)
    image_np = load_image_into_numpy_array(image)
    image_np_expanded = np.expand_dims(image_np, axis=0)
    output_dict = run_inference_for_single_image(image_np, sess, detection_graph)
    write_to_json(output_dict, im_name, first)
    first = False

file.write("]")
file.close()
sess.close()
'''vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    output_dict['detection_boxes'],
    output_dict['detection_classes'],
    output_dict['detection_scores'],
    category_index,
    instance_masks=output_dict.get('detection_masks'),
    use_normalized_coordinates=True,
    line_thickness=8)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)'''

13304

"vis_util.visualize_boxes_and_labels_on_image_array(\n    image_np,\n    output_dict['detection_boxes'],\n    output_dict['detection_classes'],\n    output_dict['detection_scores'],\n    category_index,\n    instance_masks=output_dict.get('detection_masks'),\n    use_normalized_coordinates=True,\n    line_thickness=8)\nplt.figure(figsize=IMAGE_SIZE)\nplt.imshow(image_np)"

In [44]:
'''class_labels = {'bottle': 44, 'person': 1, 'backpack': 27, 'handbag': 31, 
                'umbrella': 28, 'bowl': 51, 'cup': 47, 'clock': 85, 
                'knife': 49, 'spoon': 50, 'fork': 48, 'chair': 62, 
                'potted plant': 64, 'dining table': 67, 'keyboard': 76, 'mouse': 74, 
                'laptop': 73, 'cell phone': 77, 'scissors': 87, 'book': 84, 'monitor': 72}

class_ids = {44, 1, 27, 31, 28, 51, 47, 85, 49, 50, 48, 62, 64, 67, 76, 74, 73, 77, 87, 84, 72}
import pickle 
fileObject = open('/home/linda/OLIV/training/data/filenames_id.p','rb')  
filenames_id = pickle.load(fileObject)''' 

In [53]:
'''file = open("subsetCOCO_testRes.json", "w")
file.write("[")
i = 0 
while i < output_dict['num_detections']:
    # "image_id":1, "category_id":18, "bbox": [x, y, width, height], "score":0.7
    if output_dict['detection_classes'][i] in class_ids and output_dict['detection_scores'][i] > 0:
        if i == 0:
            file.write("{\"image_id\": " + str(filenames_id[IMAGE_NAME])
                      + ", \"category_id\": "+ str(output_dict['detection_classes'][i])
                      + ", \"bbox\": [" + str(output_dict['detection_boxes'][i][1]*image.size[0]) + ", "
                      + str(output_dict['detection_boxes'][i][0]*image.size[1]) + ", "
                      + str((output_dict['detection_boxes'][i][3]-output_dict['detection_boxes'][i][1])*image.size[0]) + ", "
                      + str((output_dict['detection_boxes'][i][2]-output_dict['detection_boxes'][i][0])*image.size[1])
                      + "], \"score\": " + str(output_dict['detection_scores'][i]) + "}")
        else:
            file.write(",{\"image_id\": " + str(filenames_id[IMAGE_NAME])
                      + ", \"category_id\": "+ str(output_dict['detection_classes'][i])
                      + ", \"bbox\": [" + str(output_dict['detection_boxes'][i][1]*image.size[0]) + ", "
                      + str(output_dict['detection_boxes'][i][0]*image.size[1]) + ", "
                      + str((output_dict['detection_boxes'][i][3]-output_dict['detection_boxes'][i][1])*image.size[0]) + ", "
                      + str((output_dict['detection_boxes'][i][2]-output_dict['detection_boxes'][i][0])*image.size[1])
                      + "], \"score\": " + str(output_dict['detection_scores'][i]) + "}")
    i += 1

file.write("]")
file.close()'''